In [2]:
import pandas as pd
players=pd.read_csv("players.csv")

In [3]:
players.head(20)

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
5,Amateur,True,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee...,0.0,Adrian,Female,17,NaN,NaN
6,Regular,True,8e594b8953193b26f498db95a508b03c6fe1c24bb5251d...,0.0,Luna,Female,19,NaN,NaN
7,Amateur,False,1d2371d8a35c8831034b25bda8764539ab7db0f6393869...,0.0,Emerson,Male,21,NaN,NaN
8,Amateur,True,8b71f4d66a38389b7528bb38ba6eb71157733df7d17403...,0.1,Natalie,Male,17,NaN,NaN
9,Veteran,True,bbe2d83de678f519c4b3daa7265e683b4fe2d814077f90...,0.0,Nyla,Female,22,NaN,NaN


In [4]:
# Check data info
players.info()

# Drop completely empty columns or duplicates if any
players.drop_duplicates()

# Filter and select relevant columns
players_tidy = players.loc[players["played_hours"] > 0, ['experience', 'subscribe', 'played_hours']]

# Display
players_tidy.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   experience        196 non-null    object 
 1   subscribe         196 non-null    bool   
 2   hashedEmail       196 non-null    object 
 3   played_hours      196 non-null    float64
 4   name              196 non-null    object 
 5   gender            196 non-null    object 
 6   age               196 non-null    int64  
 7   individualId      0 non-null      float64
 8   organizationName  0 non-null      float64
dtypes: bool(1), float64(3), int64(1), object(4)
memory usage: 12.6+ KB


,experience,subscribe,played_hours
0,Pro,True,30.3
1,Veteran,True,3.8
3,Amateur,True,0.7
4,Regular,True,0.1
8,Amateur,True,0.1


In [39]:
import altair as alt

# --- Experience facet ---
chart_exp = (
    alt.Chart(players_tidy)
    .mark_bar()
    .encode(
        x=alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Hours'),
        y=alt.Y('count()', title='Count'),
        color='experience:N'
    )
    .properties(title='Played Hours by Experience')
    .facet(column='experience:N')
)

# --- Subscribe facet ---
chart_sub = (
    alt.Chart(players_tidy)
    .mark_bar()
    .encode(
        x=alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Hours'),
        y=alt.Y('count()', title='Count'),
        color=alt.Color('subscribe:N').title("subscribed")
    )
    .properties(title='Played Hours by Subscription')
    .facet(column='subscribe:N')
)

chart_exp 

alt.FacetChart(...)

In [40]:
chart_sub

alt.FacetChart(...)

In [46]:
# --- Experience facet (zoomed) ---
chart_exp_zoom = (
    alt.Chart(players_tidy)
    .mark_bar(clip=True)
    .encode(
        x=alt.X('played_hours:Q',
                bin=alt.Bin(maxbins=45),
                scale=alt.Scale(domain=[0, 60]),   
                title='Played Hours'),
        y=alt.Y('count()', title='Count'),
        color='experience:N'
    )
    .properties(title='Played Hours by Experience (0–60)')
    .facet(column='experience:N')
)

# --- Subscribe facet (zoomed) ---
chart_sub_zoom = (
    alt.Chart(players_tidy)
    .mark_bar(clip=True)
    .encode(
        x=alt.X('played_hours:Q',
                bin=alt.Bin(maxbins=40),
                scale=alt.Scale(domain=[0, 60]), 
                title='Played Hours'),
        y=alt.Y('count()', title='Count'),
        color=alt.Color('subscribe:N').title("subscribed")
    )
    .properties(title='Played Hours by Subscription (0–60)')
    .facet(column='subscribe:N')
)

chart_exp_zoom 

alt.FacetChart(...)

In [43]:
chart_sub_zoom

alt.FacetChart(...)